Reward Function and Environment Dynamic Class(es) for Single Agent Gym

A single agent gym (SingleAgent()), named mujoco_gym in the following, enables the training of a single agent with reinforcement learning. To instaniate a mujoco_gym, among other parameters 1. the reward function and 2. the environment dynamic class(es) need to be specified. How these may be implemented is explained in the following.

1. Reward Function 

A reward function determines the reward an agent receives after an action in a gym. In our test_reward function an agent receives a reward for both 1.1 moving towards a target and 1.2 moving at all in the mujoco_gym. The final reward returned by the function is the sum of these two. 


1.1 Reward for moving towards a target

The variable distance (line 1) stores the distance between the "torso" and the "current_target". For that it uses the mujoco_gym function calculate_distance, which can handle arrays with the coordinates of an object as well as strings with the name of an object to read out this objects coordinates. 
Next, the if-else query (lines 2-7) checks whether the dictionary data_store from the mujoco_gym already has the key "distance". 
If not (lines 2-4) the entry "distance" is created with the value of the variable distance. The (new_)reward is zero in this case, as the agent never attempted to move towards the "current_target" before. 
Otherwise (lines 5-7), if "distance" already exists in the data_store, it represents the distance to the "current_target" before the last step. The difference between this former distance and the current distance is the (new_)reward. If the "torso" moved closer to the "current_target" it is positive. If the agent did not move at all it is zero, and if the agent moved further away, it is negative. Then the data_store dictionary entry "distance" is updated with the current distance.
Lastly, the variable reward (line 8) stores the new_reward to allow an easy addition later on.

In [ ]:
distance = mujoco_gym.calculate_distance("torso", mujoco_gym.data_store["current_target"])
if "distance" not in mujoco_gym.data_store.keys():
    mujoco_gym.data_store["distance"] = distance
    new_reward = 0
else:
    new_reward = mujoco_gym.data_store["distance"] - distance
    mujoco_gym.data_store["distance"] = distance
reward = new_reward

1.2 Reward for moving at all

Similarily to 1., the if-else-query (lines 1-9) checks whether the dictionary data_store from the mujoco_gym already has the key "last_position". 
If not (lines 1-3) the entry "last_position" is created with a deepcopy of the coordinates (np.ndarray) from the "torso". The (new_)reward is zero in this case, as the agent never attempted to move before. 
Otherwise (lines 4-9), if "last_position" already exists in the data_store, the agents position was checked before the last step. The (new_)reward is the distance between the agents last and its current position. It is calculated with the calculate_distance function from mujoco_gym, like the distance between the "torso" and the "current_target" in 1. Next, the data_store entry "last_position" is updated to the current position to be used in the next step. If the movement of the agent was too small, namely less than 0.08 (line 7), the agent receives a negative reward. Reward shaping revealed to multiply this (new_)reward by a factor of 6.

In [ ]:
if "last_position" not in mujoco_gym.data_store.keys():
    mujoco_gym.data_store["last_position"] = copy.deepcopy(mujoco_gym.data.body("torso").xipos)
    new_reward = 0
else:
    new_reward = mujoco_gym.calculate_distance("torso", mujoco_gym.data_store["last_position"])
    mujoco_gym.data_store["last_position"] = copy.deepcopy(mujoco_gym.data.body("torso").xipos)
    if new_reward < 0.08:
        new_reward = new_reward * -1
    new_reward = new_reward * 6

Lastly, the return variable reward is the sum of the reward for moving towards the "current target" and the new_reward for moving at all.

In [ ]:
reward = reward + new_reward

2. Environment Dynamic Class(es)

The Environment Dynamics enable updating features of the data store and giving out an additional reward on special occassions. They are written in separate classes to set boundaries for the observation space and thus enable normalizing the observations more precicely than by normalizing with infinity. So far there are two Environment Dynamics: 2.1 Environment Dynamic and 2.2 Pick Up Dynamic. 

2.1 Environment Dynamic Class

2.1.1 Attributes 

The Environment Dynamics receive a mujoco_gym as parameter. Besides this, it has an observation space. That is, a dicitonary which contains the lowest and highest values of the return variable target_coordinates to normalize it.

2.1.2 dynamic Method
The function dynamic updates the target if an agent is close enough. 

Therefore, it first checks with an if query (lines 1-3), whether the data store of an agent already contains targets. If not, it adds the entry "targets" to the data_store dictionary with every object in the environment which has the tag "target", this excludes e.g. the agent or walls. Afterwards, a random target is selected to be the current target for the agent.

In [ ]:
if "targets" not in self.mujoco_gym.data_store.keys():
    self.mujoco_gym.data_store["targets"] = self.mujoco_gym.filterByTag("target")
    self.mujoco_gym.data_store["current_target"] = self.mujoco_gym.data_store["targets"][random.randint(0, len(self.mujoco_gym.data_store["targets"]) - 1)]["name"]

As soon as the agent has a current target, the distance between the agent and its target is calculated. Therefore, we us the same function like in the reward funciton (see above). If the agent is close to the target (lines 2-3), a new target is selected and the distance to the new target is calculated and stored in the data store. The SingleAgent class requires each Environment Dynamic class to return a reward, which is useful in some cases (see 2.2). As this feature is not used here, it is simply set to 0. Lastly, the coordinates of the current target are read out from the data store and together with the 0 reward returned by the function.

In [ ]:
distance = self.mujoco_gym.calculate_distance("torso", self.mujoco_gym.data_store["current_target"])
if distance < 1:
    self.mujoco_gym.data_store["current_target"] = self.mujoco_gym.data_store["targets"][random.randint(0, len(self.mujoco_gym.data_store["targets"]) - 1)]
    self.mujoco_gym.data_store["distance"] = self.mujoco_gym.calculate_distance("torso", self.mujoco_gym.data_store["current_target"])
reward = 0
target_coordinates = self.mujoco_gym.data.body(self.mujoco_gym.data_store["current_target"]).xipos
return reward, target_coordinates

2.2 Pick Up Dynamic Class

2.2.1 Attributes
The attributes are quite similar to the Environment Dynamic Class. The only difference lies in the additional entry in the values of the observation space. This entry tells whether an agent carries an object or not. 

2.2.2 dynamic Method 
The function dynamic updates the target if an agent is close enough as well as adds or removes an inventory to or from the agent as an observation.

First, the method checks whether all needed keys are in the data store. Therefore, it checks if "inventory" is in the data store (lines 1-2) and adds it with a list containing only a 0 if neccessary. Furthermore, it checks whether the data store has "targets" (lines 3-5). Occasionally it sets the values to a list of all objects tagged as target and randomly chooses one of them as current target. 

In [ ]:
if "inventory" not in self.mujoco_gym.data_store.keys():
    self.mujoco_gym.data_store["inventory"] = [0]
if "targets" not in self.mujoco_gym.data_store.keys():
    self.mujoco_gym.data_store["targets"] = self.mujoco_gym.filterByTag("target")
    self.mujoco_gym.data_store["current_target"] = self.mujoco_gym.data_store["targets"][random.randint(0, len(self.mujoco_gym.data_store["targets"]) - 1)]["name"]

Next, the distance between the agent and the current target is calculated. If the agent is close to the target (lines 1-10), the target is reached and the reward feature of the Environment Dynamic comes into play. An if-elif query, checks whether the agents inventory is containing an object. If not (lines 3-5), the agent picks up the target by incrementing the inventory and receives a reward. Otherwise (lines 6-8), the agent drops its inventory by setting the value to 0 and receives a reward. Thereafter a new target is randomly chosen and the distance to the new target is calculated and stored in the data store. 
Lastly, the coordinates of the target and the inventory are concatenated to one array and returned together with the reward.

In [ ]:
distance = self.mujoco_gym.calculate_distance("torso", self.mujoco_gym.data_store["current_target"])
if distance < 2:
    print("target reached")
    if self.mujoco_gym.data_store["inventory"][0] == 0:
        self.mujoco_gym.data_store["inventory"][0] = 1
        reward = 1
    elif self.mujoco_gym.data_store["inventory"][0] == 1:
        self.mujoco_gym.data_store["inventory"][0] = 0
        reward = 1
    self.mujoco_gym.data_store["current_target"] = self.mujoco_gym.data_store["targets"][random.randint(0, len(self.mujoco_gym.data_store["targets"]) - 1)]["name"]
    self.mujoco_gym.data_store["distance"] = self.mujoco_gym.calculate_distance("torso", self.mujoco_gym.data_store["current_target"])
current_target_coordinates_with_inventory = np.concatenate((self.mujoco_gym.data.body(self.mujoco_gym.data_store["current_target"]).xipos, self.mujoco_gym.data_store["inventory"]))
return reward, current_target_coordinates_with_inventory